# Phi-4 Mini ONNX 并行函数调用教程

本教程展示了如何使用 Phi-4 Mini 和 ONNX Runtime GenAI 进行并行函数调用。函数调用使模型能够根据用户请求智能地调用外部工具和 API。

## 概述

在本教程中，您将学习如何：
- 设置 Phi-4 Mini 和 ONNX Runtime GenAI
- 定义用于预订航班和酒店的函数架构
- 使用 Lark 语法进行引导生成以获得结构化输出
- 执行并行函数调用以处理复杂的旅行预订场景

## 前提条件

在运行本教程之前，请确保您已完成以下操作：
- 下载 Phi-4 Mini ONNX 模型
- 安装 `onnxruntime-genai` 包
- 对函数调用概念有基本了解


## 第一步：导入所需库

首先，我们将导入实现函数调用所需的库。


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## 第2步：模型设置与配置

现在我们将配置 Phi-4 Mini ONNX 模型。请确保将路径替换为您的实际模型目录。


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## 第三步：配置生成参数

设置生成参数以控制模型的输出行为。这些设置可确保函数调用的响应具有确定性和针对性。


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## 第四步：定义可用函数

在这里，我们定义了 AI 助手可以调用的函数。在这个例子中，我们有两个与旅行相关的函数：
1. **booking_flight_tickets**：用于预订机场之间的航班
2. **booking_hotels**：用于预订酒店住宿

函数定义遵循 OpenAI 的函数调用模式格式。


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## 第五步：语法生成辅助函数

这些辅助函数将我们的函数定义转换为 Lark 语法格式，ONNX Runtime GenAI 使用该格式进行引导生成。这可以确保模型以正确的 JSON 格式输出有效的函数调用。


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## 第六步：测试语法生成

让我们测试语法生成函数，看看它们如何将工具定义转换为正确的格式。


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## 第七步：准备系统提示和生成器

现在我们将创建系统提示，告知模型可用的工具，并设置生成器的引导生成参数。


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## 第八步：使用引导生成初始化生成器

现在我们将使用配置的参数创建生成器，并应用 Lark 语法进行引导生成。


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## 第九步：测试并行函数调用

现在，让我们通过一个复杂的旅行预订场景来测试我们的设置，该场景需要调用多个函数。


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### 生成函数调用

模型现在将根据用户请求生成结构化的函数调用。通过引导生成，输出将是有效的 JSON 格式，可以直接执行。

**预期输出**：模型应生成以下函数调用：
1. `booking_flight_tickets`，包含从北京 (PEK) 到巴黎 (CDG) 的航班信息
2. `booking_hotels`，包含巴黎住宿信息

运行下面的单元格以查看实时生成：


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## 结论

🎉 **恭喜你！** 你已经成功使用 ONNX Runtime GenAI 实现了 Phi-4 Mini 的并行函数调用。

### 你学到了什么：

1. **模型配置**：如何使用 ONNX Runtime GenAI 配置 Phi-4 Mini
2. **函数定义**：如何定义 AI 函数调用的函数架构
3. **引导生成**：如何使用 Lark 语法生成结构化输出
4. **并行函数调用**：如何处理需要多个函数调用的复杂请求

### 主要优势：

- ✅ **结构化输出**：引导生成确保有效的 JSON 函数调用
- ✅ **并行处理**：在单个请求中处理多个函数调用
- ✅ **高性能**：ONNX Runtime 提供优化的推理性能
- ✅ **灵活架构**：轻松添加或修改函数定义

### 资源：

- [Phi-4 Mini 文档](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [ONNX Runtime GenAI 文档](https://onnxruntime.ai/docs/genai/)
- [函数调用最佳实践](https://platform.openai.com/docs/guides/function-calling)



---

**免责声明**：  
本文档使用AI翻译服务[Co-op Translator](https://github.com/Azure/co-op-translator)进行翻译。尽管我们努力确保翻译的准确性，但请注意，自动翻译可能包含错误或不准确之处。原始语言的文档应被视为权威来源。对于关键信息，建议使用专业人工翻译。我们不对因使用此翻译而产生的任何误解或误读承担责任。
